In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Global plot settings
plt.rcParams['figure.dpi'] = 150   # Set the resolution of the plots to 150 DPI
plt.style.use('fivethirtyeight')

In [2]:
df_sales = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/04_featured_data_with_weather_variables.csv', encoding='utf-8')


df_sales.head()

,id,Datum,Warengruppe,Umsatz,KielerWoche,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,Woche,Monat,Wochentag,Feiertag,Ferien,sunny,cloudy,rainy,thunderstorm
0,1307011,2013-07-01,1,148.828353,0,6,17.8375,15,cloudy,27,7,1,0,0,0,1,0,0
1,1307012,2013-07-01,2,535.856285,0,6,17.8375,15,cloudy,27,7,1,0,0,0,1,0,0
2,1307013,2013-07-01,3,201.198426,0,6,17.8375,15,cloudy,27,7,1,0,0,0,1,0,0
3,1307014,2013-07-01,4,65.890169,0,6,17.8375,15,cloudy,27,7,1,0,0,0,1,0,0
4,1307015,2013-07-01,5,317.475875,0,6,17.8375,15,cloudy,27,7,1,0,0,0,1,0,0
